In [1]:
import numpy as np
np.set_printoptions(precision=1)
import tensorflow as tf
import matplotlib.pylab as plt

from modules.utils import load_cifar10
from modules.cnn_with_spectral_parameterization import CNN_Spectral_Param
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool

% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
xtrain, ytrain, xtest, ytest = load_cifar10(5, channels_last=True)

file already downloaded..
getting batch 1
getting batch 2
getting batch 3
getting batch 4
getting batch 5


In [3]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [19]:
# take a small batch for testing architecture
sampleind = np.random.choice(np.arange(xtrain.shape[0]),
                             size=8, replace=False)
xtrain_sample = xtrain[sampleind]
ytrain_sample = ytrain[sampleind]
# assert len(np.unique(ytrain_sample)) == 10
print(np.bincount(ytrain_sample))

[0 1 1 0 1 1 1 2 0 1]


In [5]:
def write_error_rate(name, error_rates):
    with open('error_rates/' + name + '.txt', 'w') as f:
        for err in error_rates:
            f.write("%s\n" % err)

# Generic Architecture

In [16]:
tf.reset_default_graph()
generic_3_spectral = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='generic', use_spectral_params=False)
generic_3_spectral.train(xtrain_sample, ytrain_sample, None, None,
          batch_size=128, epochs=150)

write_error_rate('generic_3_spectral', generic_3_spectral.error_rate)

number of batches for training: 1
epoch 1 
Error rate: 0.9375
Train acc: 0.0625
Loss: 3.45112
epoch 2 
Error rate: 0.84375
Train acc: 0.15625
Loss: 3.39278
epoch 3 
Error rate: 0.84375
Train acc: 0.15625
Loss: 3.35777
epoch 4 
Error rate: 0.84375
Train acc: 0.15625
Loss: 3.34999
epoch 5 
Error rate: 0.84375
Train acc: 0.15625
Loss: 3.33067
epoch 6 
Error rate: 0.8203125
Train acc: 0.1796875
Loss: 3.31778
epoch 7 
Error rate: 0.78125
Train acc: 0.21875
Loss: 3.29782
epoch 8 
Error rate: 0.7109375
Train acc: 0.2890625
Loss: 3.28828
epoch 9 
Error rate: 0.6484375
Train acc: 0.3515625
Loss: 3.25761
epoch 10 
Error rate: 0.671875
Train acc: 0.328125
Loss: 3.25127
epoch 11 
Error rate: 0.65625
Train acc: 0.34375
Loss: 3.21794
epoch 12 
Error rate: 0.6640625
Train acc: 0.3359375
Loss: 3.20619
epoch 13 
Error rate: 0.6640625
Train acc: 0.3359375
Loss: 3.15712
epoch 14 
Error rate: 0.6796875
Train acc: 0.3203125
Loss: 3.14854
epoch 15 
Error rate: 0.609375
Train acc: 0.390625
Loss: 3.10107
epoc

Error rate: 0.0625
Train acc: 0.9375
Loss: 1.29252
epoch 130 
Error rate: 0.140625
Train acc: 0.859375
Loss: 1.43166
epoch 131 
Error rate: 0.046875
Train acc: 0.953125
Loss: 1.27497
epoch 132 
Error rate: 0.0625
Train acc: 0.9375
Loss: 1.38772
epoch 133 
Error rate: 0.0390625
Train acc: 0.9609375
Loss: 1.23858
epoch 134 
Error rate: 0.1015625
Train acc: 0.8984375
Loss: 1.38166
epoch 135 
Error rate: 0.046875
Train acc: 0.953125
Loss: 1.25058
epoch 136 
Error rate: 0.078125
Train acc: 0.921875
Loss: 1.38126
epoch 137 
Error rate: 0.03125
Train acc: 0.96875
Loss: 1.2239
epoch 138 
Error rate: 0.078125
Train acc: 0.921875
Loss: 1.35652
epoch 139 
Error rate: 0.03125
Train acc: 0.96875
Loss: 1.21352
epoch 140 
Error rate: 0.0546875
Train acc: 0.9453125
Loss: 1.35038
epoch 141 
Error rate: 0.0390625
Train acc: 0.9609375
Loss: 1.21571
epoch 142 
Error rate: 0.1328125
Train acc: 0.8671875
Loss: 1.37495
epoch 143 
Error rate: 0.0234375
Train acc: 0.9765625
Loss: 1.20861
epoch 144 
Error rate:

In [6]:
tf.reset_default_graph()
generic_3_spatial = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='generic', use_spectral_params=False)
generic_3_spatial.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('generic_3_spatial', generic_3_spatial.error_rate)

number of batches for training: 390
epoch 1 
0.431750801282
2.18963
epoch 2 
0.55608974359
1.76338
epoch 3 
0.606891025641
1.61115
epoch 4 
0.626862980769
1.53891
epoch 5 
0.666927083333
1.42805
epoch 6 
0.667728365385
1.4197
epoch 7 
0.697796474359
1.33081
epoch 8 
0.695012019231
1.32748
epoch 9 
0.725881410256
1.23733
epoch 10 
0.712099358974
1.2688
epoch 11 
0.746834935897
1.17087
epoch 12 
0.727023237179
1.20838
epoch 13 
0.756911057692
1.12436
epoch 14 
0.752123397436
1.13705
epoch 15 
0.775420673077
1.06535
epoch 16 
0.769631410256
1.07902
epoch 17 
0.784174679487
1.03028
epoch 18 
0.784174679487
1.03549
epoch 19 
0.801983173077
0.981741
epoch 20 
0.78733974359
1.0166
epoch 21 
0.817728365385
0.927242
epoch 22 
0.792608173077
0.998451
epoch 23 
0.830408653846
0.889879
epoch 24 
0.815604967949
0.92683
epoch 25 
0.826342147436
0.899272
epoch 26 
0.81891025641
0.917293
epoch 27 
0.85
0.831039
epoch 28 
0.830108173077
0.881782
epoch 29 
0.844330929487
0.841295
epoch 30 
0.83323317307

In [7]:
tf.reset_default_graph()
generic_5_spectral = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=True)
generic_5_spectral.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('generic_5_spectral', generic_5_spectral.error_rate)

number of batches for training: 390
epoch 1 
0.406149839744
2.38769
epoch 2 
0.524178685897
1.9143
epoch 3 
0.567147435897
1.76323
epoch 4 
0.574959935897
1.725
epoch 5 
0.62359775641
1.57476
epoch 6 
0.625881410256
1.55206
epoch 7 
0.652844551282
1.47679
epoch 8 
0.64937900641
1.46121
epoch 9 
0.683814102564
1.35854
epoch 10 
0.669050480769
1.38611
epoch 11 
0.704186698718
1.2773
epoch 12 
0.689302884615
1.30459
epoch 13 
0.725741185897
1.19808
epoch 14 
0.710236378205
1.22859
epoch 15 
0.751362179487
1.10628
epoch 16 
0.71344150641
1.20082
epoch 17 
0.739483173077
1.12392
epoch 18 
0.726101762821
1.14859
epoch 19 
0.767608173077
1.03318
epoch 20 
0.727684294872
1.13582
epoch 21 
0.778625801282
0.995855
epoch 22 
0.762780448718
1.03135
epoch 23 
0.802363782051
0.925459
epoch 24 
0.778265224359
0.985748
epoch 25 
0.825240384615
0.86337
epoch 26 
0.794931891026
0.938685
epoch 27 
0.779827724359
0.983136
epoch 28 
0.799819711538
0.924101
epoch 29 
0.849579326923
0.798184
epoch 30 
0.8213

In [8]:
tf.reset_default_graph()
generic_5_spatial = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=False)
generic_5_spatial.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('generic_5_spatial', generic_5_spatial.error_rate)

number of batches for training: 390
epoch 1 
0.406029647436
2.24496
epoch 2 
0.531550480769
1.83483
epoch 3 
0.586598557692
1.66671
epoch 4 
0.615625
1.57902
epoch 5 
0.641666666667
1.5051
epoch 6 
0.657592147436
1.45847
epoch 7 
0.680789262821
1.3864
epoch 8 
0.688541666667
1.3577
epoch 9 
0.711358173077
1.29648
epoch 10 
0.708072916667
1.29351
epoch 11 
0.734134615385
1.21566
epoch 12 
0.728545673077
1.22058
epoch 13 
0.74781650641
1.16856
epoch 14 
0.747936698718
1.156
epoch 15 
0.774619391026
1.08347
epoch 16 
0.752163461538
1.13355
epoch 17 
0.782451923077
1.04964
epoch 18 
0.770973557692
1.07222
epoch 19 
0.800020032051
0.99195
epoch 20 
0.778946314103
1.03881
epoch 21 
0.811818910256
0.950453
epoch 22 
0.781530448718
1.0324
epoch 23 
0.820172275641
0.921175
epoch 24 
0.793249198718
0.985058
epoch 25 
0.831009615385
0.879662
epoch 26 
0.805969551282
0.952114
epoch 27 
0.842467948718
0.84406
epoch 28 
0.826241987179
0.883395
epoch 29 
0.865044070513
0.781019
epoch 30 
0.8178685897

# Deep Architecture

In [20]:
tf.reset_default_graph()
deep_3_spatial = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='deep', 
                                    use_spectral_params=False, learning_rate=1e-4)
deep_3_spatial.train(xtrain_sample, ytrain_sample, None, None,
          batch_size=128, epochs=150)

write_error_rate('deep_3_spatial', deep_3_spatial.error_rate)

number of batches for training: 1
epoch 1 
Error rate: 1.0
Train acc: 0.0
Loss: 2.31271
epoch 2 
Error rate: 0.875
Train acc: 0.125
Loss: 2.30196
epoch 3 
Error rate: 0.875
Train acc: 0.125
Loss: 2.29361
epoch 4 
Error rate: 0.875
Train acc: 0.125
Loss: 2.29165
epoch 5 
Error rate: 0.875
Train acc: 0.125
Loss: 2.2857
epoch 6 
Error rate: 0.625
Train acc: 0.375
Loss: 2.27843
epoch 7 
Error rate: 0.75
Train acc: 0.25
Loss: 2.2727
epoch 8 
Error rate: 0.75
Train acc: 0.25
Loss: 2.26644
epoch 9 
Error rate: 0.75
Train acc: 0.25
Loss: 2.25475
epoch 10 
Error rate: 0.75
Train acc: 0.25
Loss: 2.2419
epoch 11 
Error rate: 0.75
Train acc: 0.25
Loss: 2.23247
epoch 12 
Error rate: 0.75
Train acc: 0.25
Loss: 2.22288
epoch 13 
Error rate: 0.75
Train acc: 0.25
Loss: 2.20189
epoch 14 
Error rate: 0.75
Train acc: 0.25
Loss: 2.17861
epoch 15 
Error rate: 0.75
Train acc: 0.25
Loss: 2.16387
epoch 16 
Error rate: 0.75
Train acc: 0.25
Loss: 2.15156
epoch 17 
Error rate: 0.75
Train acc: 0.25
Loss: 2.11882
e

Error rate: 0.0
Train acc: 1.0
Loss: 0.00919155
epoch 160 
Error rate: 0.0
Train acc: 1.0
Loss: 0.040777
epoch 161 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00638671
epoch 162 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0203601
epoch 163 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0114985
epoch 164 
Error rate: 0.0
Train acc: 1.0
Loss: 0.029155
epoch 165 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00730681
epoch 166 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0134332
epoch 167 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0050884
epoch 168 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0194774
epoch 169 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00523485
epoch 170 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0079777
epoch 171 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00685963
epoch 172 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0197639
epoch 173 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00593924
epoch 174 
Error rate: 0.0
Train acc: 1.0
Loss: 0.0132681
epoch 175 
Error rate: 0.0
Train acc: 1.0
Loss: 0.00744083
epoch 176 
Error rat

In [ ]:
tf.reset_default_graph()
deep_3_spectral = CNN_Spectral_Param(num_output=10, kernel_size=3, architecture='deep', use_spectral_params=True)
deep_3_spectral.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('deep_3_spectral', deep_3_spectral.error_rate)

number of batches for training: 390
epoch 1 
0.260677083333
6.97993
epoch 2 
0.387800480769
6.12953
epoch 3 
0.451482371795
5.63513
epoch 4 
0.514362980769
5.23542
epoch 5 
0.554547275641
4.96855
epoch 6 
0.576422275641
4.7862
epoch 7 
0.605428685897
4.60694
epoch 8 
0.608774038462
4.50032
epoch 9 
0.626141826923
4.3792
epoch 10 
0.63265224359
4.30027
epoch 11 
0.651923076923
4.19223
epoch 12 
0.65094150641
4.14057
epoch 13 
0.663241185897
4.06442
epoch 14 
0.65999599359
4.02599
epoch 15 
0.681049679487
3.93184
epoch 16 
0.680288461538
3.90053
epoch 17 
0.695893429487
3.82474
epoch 18 
0.685857371795
3.81255
epoch 19 
0.705248397436
3.7299
epoch 20 
0.688721955128
3.74192
epoch 21 
0.70703125
3.66772
epoch 22 
0.706891025641
3.64212
epoch 23 
0.715705128205
3.59106
epoch 24 
0.709114583333
3.58332
epoch 25 
0.725961538462
3.51617
epoch 26 
0.717548076923
3.51518
epoch 27 
0.731530448718
3.45399
epoch 28 
0.725
3.45231
epoch 29 
0.738601762821
3.39186
epoch 30 
0.73046875
3.39165
epoch 

In [ ]:
tf.reset_default_graph()
deep_5_spatial = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='deep', use_spectral_params=False)
deep_5_spatial.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('deep_5_spatial', deep_5_spatial.error_rate)

In [ ]:
tf.reset_default_graph()
deep_5_spectral = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='deep', use_spectral_params=True)
deep_5_spectral.train(xtrain, ytrain, None, None,
          batch_size=128, epochs=150)

write_error_rate('deep_5_spectral', deep_5_spectral.error_rate)

# Spectral Pooling Architecture

In [22]:
xtrain_NCHW, ytrain_NCHW, xtest_NCHW, ytest_NCHW = load_cifar10(5, channels_last=False)

sampleind_NCHW = np.random.choice(np.arange(xtrain_NCHW.shape[0]),
                             size=64, replace=False)
xtrain_sample_NCHW = xtrain_NCHW[sampleind_NCHW]
ytrain_sample_NCHW = ytrain_NCHW[sampleind_NCHW]
assert len(np.unique(ytrain_sample_NCHW)) == 10
print(np.bincount(ytrain_sample_NCHW))

tf.reset_default_graph()
cnn3 = CNN_Spectral_Pool()

cnn3.train(xtrain_sample_NCHW,
          ytrain_sample_NCHW,
          xtrain_sample_NCHW,
          ytrain_sample_NCHW,
          batch_size=64,
          epochs=100,
          extra_conv_layer=True,
          use_global_averaging=True)

file already downloaded..
getting batch 1
getting batch 2
getting batch 3
getting batch 4
getting batch 5
[ 6  6  2 10  4  8  6  6  9  7]


TypeError: train() got an unexpected keyword argument 'verbose'